In [1]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler, MinMaxScaler, QuantileTransformer
from sklearn.model_selection import GroupShuffleSplit, train_test_split
from sklearn.pipeline import Pipeline
from os.path import join, exists
from tqdm import tqdm

import pandas as pd
import numpy as np
import traceback
import random
import pickle
import glob
import json
import yaml
import time
import copy
import re
import os

from multiprocessing import Pool, cpu_count

from holodecml.vae.checkpointer import *
from holodecml.vae.optimizers import *
from holodecml.vae.tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import *
import torch.nn.functional as F
from torch import nn
import torch

import matplotlib.pyplot as plt

In [2]:
is_cuda = torch.cuda.is_available()
device = torch.device(torch.cuda.current_device()) if is_cuda else torch.device("cpu")

if is_cuda:
    torch.backends.cudnn.benchmark = True

print(f'Preparing to use device {device}')

Preparing to use device cuda:0


### Load the configuration

In [3]:
with open("results/9_29/exp1/config.yml") as config_file:
    config = yaml.load(config_file, Loader=yaml.FullLoader)

In [4]:
class LoadGeckoPandasExperiment:
    
    def __init__(self, 
                 data_path, 
                 input_cols, 
                 output_cols, 
                 experiment_subset = False,
                 x_data = False,
                 y_data = False,
                 shuffle = True):
        
        self.data_path = data_path
        self.input_cols = input_cols
        self.output_cols = output_cols  
        self.experiment_subset = experiment_subset
        self.x = x_data
        self.y = y_data
        
        self.shuffle = shuffle
        self.load()
    
    def load(self):
        if not isinstance(self.x, pd.DataFrame):
            self.x = pd.read_csv(os.path.join(self.data_path, "row_data_in.csv"))
            self.x["id"] = self.x["id"].apply(lambda x: int(x.strip("Exp")))
        if not isinstance(self.y, pd.DataFrame):
            self.y = pd.read_csv(os.path.join(self.data_path, "row_data_out.csv"))
            self.y["id"] = self.y["id"].apply(lambda x: int(x.strip("Exp")))
        
        drop_cols = ["index", "indexer"]
        for df in [self.x, self.y]:
            keep_cols = [x for x in df.columns if x not in drop_cols]
            df = df[keep_cols].copy()
            
        if self.experiment_subset:
            cond1 = self.x['id'].isin(self.experiment_subset)
            cond2 = self.y['id'].isin(self.experiment_subset)
            self.x = self.x[cond1].copy()
            self.y = self.y[cond2].copy()
        
        self.x_dict = {
            experiment: group for (experiment, group) in self.x.groupby(["id"])
        }
        self.y_dict = {
            experiment: group for (experiment, group) in self.y.groupby(["id"])
        }
        self.w_dict = {
            experiment: group["weight"] for (experiment, group) in self.y_dict.items()
        }
        
        with open(os.path.join(self.data_path, "scalers.pkl"), "rb") as fid:
            self.scaler_x, self.scaler_y = pickle.load(fid)
               
        self.x_dict = {
            experiment: self.scaler_x.transform(
                group[self.input_cols].drop(['Time [s]', 'id'], axis=1)
            ) 
            for (experiment, group) in self.x_dict.items()
        }
        self.y_dict = {
            experiment: self.scaler_y.transform(
                group[self.output_cols].drop(['Time [s]', 'id'], axis=1)
            ) 
            for (experiment, group) in self.y_dict.items()
        }
        
        self.indices = list(self.y_dict.keys())
        self.on_epoch_end()
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        idx = self.indices[idx]
        X = self.x_dict[idx]
        Y = self.y_dict[idx]
        W = self.w_dict[idx]
        
        for (x, y, w) in zip(X, Y, W):
            x_data = torch.from_numpy(x.astype(np.float32))
            y_data = torch.from_numpy(y.astype(np.float32))
            w_data = torch.from_numpy(np.array([w]))
            yield (x_data, y_data, w_data)
            
        self.processed += 1
        if (self.processed == self.__len__()):
            self.on_epoch_end()
            
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.processed = 0
        if self.shuffle == True:
            random.shuffle(self.indices)

### Create the train, test, val splits

In [5]:
random.seed(5000)

In [6]:
experiment_data = LoadGeckoPandasExperiment(
    "clustered/row_data", 
    config["data"]["input_cols"],
    config["data"]["output_cols"]
)

In [7]:
fn = "clustered/experiment_data/experiment_train_test_val_splits.pkl"
with open(fn, "rb") as fid:
    train, valid, test = pickle.load(fid)

In [8]:
train_data_set = LoadGeckoPandasExperiment(
    "clustered/row_data", 
    config["data"]["input_cols"],
    config["data"]["output_cols"],
    experiment_subset = train,
    x_data = experiment_data.x,
    y_data = experiment_data.y,
    shuffle = config["data"]["shuffle"]
)

In [9]:
valid_data_set = LoadGeckoPandasExperiment(
    "clustered/row_data", 
    config["data"]["input_cols"],
    config["data"]["output_cols"],
    experiment_subset = valid,
    x_data = experiment_data.x,
    y_data = experiment_data.y,
    shuffle = False
)

In [10]:
test_data_set = LoadGeckoPandasExperiment(
    "clustered/row_data", 
    config["data"]["input_cols"],
    config["data"]["output_cols"],
    experiment_subset = test,
    x_data = experiment_data.x,
    y_data = experiment_data.y,
    shuffle = False
)

### Initialize iterators

In [11]:
#logging.info(f"Loading training data iterator using {config['iterator']['num_workers']} workers")
    
# train_dataloader = DataLoader(
#     train_data_set,
#     **config["iterator"]
# )

# valid_dataloader = DataLoader(
#     valid_data_set,
#     **config["iterator"]
# )

# test_dataloader = DataLoader(
#     test_data_set,
#     **config["iterator"]
# )

### Model

In [12]:
class DenseNet(nn.Module):

    def __init__(self, 
                 input_size, 
                 output_size, 
                 hidden_dims = [100, 50], 
                 dropouts = [0.2, 0.2]):
        
        super(DenseNet, self).__init__()
        
        self.input_size = input_size
        self.output_size = output_size
        
        #self.embedding = nn.Embedding(train_data_set.num_timesteps, 16)
        
        self.model = [
            nn.Linear(input_size, hidden_dims[0]),
            #nn.BatchNorm1d(num_features=hidden_dims[0]),
            nn.LeakyReLU()
        ]
        if len(hidden_dims) > 1:
            if dropouts[0] > 0.0:
                self.model.append(nn.Dropout(dropouts[0]))
            for i in range(len(hidden_dims)-1):
                self.model.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
                #self.model.append(nn.BatchNorm1d(num_features=hidden_dims[i+1]))
                self.model.append(nn.LeakyReLU())
                if dropouts[i+1] > 0.0:
                    self.model.append(nn.Dropout(dropouts[i+1]))
        self.model.append(nn.Linear(hidden_dims[-1], output_size))
        self.model.append(nn.Sigmoid())
        self.model = nn.Sequential(*self.model)

    def forward(self, x):
        #x1, x2 = x
        #x1 = self.embedding(x1)
        #x = torch.cat([x1, x2], 1)
        x = self.model(x)
        return x

In [13]:
model = DenseNet(**config["model"])

In [14]:
model

DenseNet(
  (model): Sequential(
    (0): Linear(in_features=35, out_features=10000, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=10000, out_features=29, bias=True)
    (3): Sigmoid()
  )
)

In [15]:
model_dict = torch.load("results/9_29/row1/best.pt", 
                        map_location=lambda storage, loc: storage)

model.load_state_dict(model_dict["model_state_dict"])

<All keys matched successfully>

In [16]:
if is_cuda:
    model = model.to(device)

In [17]:
optimizer = LookaheadDiffGrad(model.parameters(),
                              lr=config["optimizer"]["lr"],
                              weight_decay=0.02)

In [18]:
class Loss:
    
    def __init__(self, alpha = 1.0):
        self.alpha = alpha
    
    def WMSE(self, y_true, y_pred, weights = []):   
        mse = torch.mean(weights * (y_true - y_pred) ** 2)
        return mse
        
    def train(self, y_true, y_pred, weights):   
        y_true = torch.stack(y_true).permute(1,0,2)
        y_pred = torch.stack(y_pred).permute(1,0,2)
        weights = torch.stack(weights).permute(1,0)
        
        y_true_precursor = y_true[:, :, 0]
        y_pred_precursor = y_pred[:, :, 0]

        y_true_gas = y_true[:, :, 1:15]
        y_pred_gas = y_pred[:, :, 1:15]

        y_true_aero = y_true[:, :, 15:]
        y_pred_aero = y_pred[:, :, 15:]

        mse_precursor = self.WMSE(
            y_true_precursor, 
            y_pred_precursor, 
            weights
        )
        mse_gas = self.WMSE(
            y_true_gas.permute(1, 0, 2), 
            y_pred_gas.permute(1, 0, 2),
            weights.permute(1, 0)
        )
        mse_aero = self.WMSE(
            y_true_aero.permute(1, 0, 2), 
            y_pred_aero.permute(1, 0, 2), 
            weights.permute(1, 0)
        )
        mse = mse_precursor + mse_gas + mse_aero

        kld_gas = nn.KLDivLoss()(
            F.log_softmax(y_pred_gas),
            F.softmax(y_true_gas)
        )
        kld_aero = nn.KLDivLoss()(
            F.log_softmax(y_pred_aero),
            F.softmax(y_true_aero)
        )
        return mse + self.alpha * (kld_gas + kld_aero)
    
    def test(self, y_true, y_pred):  
        y_true = torch.stack(y_true).permute(1,0,2)
        y_pred = torch.stack(y_pred).permute(1,0,2)
        
        y_true_precursor = y_true[:, :, 0]
        y_pred_precursor = y_pred[:, :, 0]

        y_true_gas = y_true[:, :, 1:15]
        y_pred_gas = y_pred[:, :, 1:15]

        y_true_aero = y_true[:, :, 15:]
        y_pred_aero = y_pred[:, :, 15:]

        mse_precursor = nn.MSELoss()(y_true_precursor, y_pred_precursor)
        mse_gas = nn.MSELoss()(y_true_gas, y_pred_gas)
        mse_aero = nn.MSELoss()(y_true_aero, y_pred_aero)
        mse = mse_precursor + mse_gas + mse_aero

        kld_gas = nn.KLDivLoss()(
            F.log_softmax(y_pred_gas),
            F.softmax(y_true_gas)
        )
        kld_aero = nn.KLDivLoss()(
            F.log_softmax(y_pred_aero),
            F.softmax(y_true_aero)
        )
        return mse + self.alpha * (kld_gas + kld_aero)

In [20]:
# # Measure allocated memory after the call
# torch.cuda.synchronize()
# end_max_memory = torch.cuda.max_memory_allocated() / 1024**2
# end_memory = torch.cuda.memory_allocated() / 1024**2

class BaseTrainer:
    
    def __init__(self, 
                 model, 
                 optimizer,
                 train_gen, 
                 valid_gen, 
                 dataloader, 
                 valid_dataloader,
                 start_epoch = 0,
                 epochs = 100,
                 timesteps = 1439,
                 window_size = 10,
                 teacher_force = True,
                 gamma = 0.5,
                 device = "cpu",
                 clip = 1.0,
                 path_save = "./"):
        
        self.model = model
        self.outsize = model.output_size
        self.optimizer = optimizer
        self.train_gen = train_gen
        self.valid_gen = valid_gen
        self.dataloader = dataloader
        self.valid_dataloader = valid_dataloader
        self.batch_size = 1 #dataloader.batch_size
        self.path_save = path_save
        self.device = device
        
        self.start_epoch = start_epoch 
        self.epochs = epochs
        self.window_size = window_size
        
        self.teacher_force = teacher_force
        self.gamma = gamma
        
        #self.criterion = nn.MSELoss()
        
        timesteps = timesteps
        self.time_range = list(range(timesteps))
                
        # Gradient clipping through hook registration
        for p in self.model.parameters():
            p.register_hook(lambda grad: torch.clamp(grad, -clip, clip))
        logger.info(f"Clipping gradients to range [-{clip}, {clip}]")
        
        # Create the save directory if it does not exist
        try:
            os.makedirs(path_save)
        except:
            pass
        
        self.criterion = Loss()
        
    def train_one_epoch(self, epoch, steps = 100):
        
        self.model.train()
        batches_per_epoch = int(np.ceil(self.train_gen.__len__() / self.batch_size))
        
        if batches_per_epoch > steps:
            batches_per_epoch = steps
            
        experiments = range(len(self.train_gen.experiment_subset))        
        batch_group_generator = tqdm(
            enumerate(experiments),
            total=batches_per_epoch, 
            leave=True
        )
    
        cost = self.tf_annealer(epoch)
        epoch_losses = {"loss": []}
        
        #model_clone = copy.deepcopy(self.model)
        for batch_idx, exp in batch_group_generator:
            y_true, y_pred, weights = [], [], []
            for i, (x,y,w) in enumerate(self.train_gen.__getitem__(exp)):
                x = x.view(1, x.size(0)).to(device)
                y = y.view(1, y.size(0)).to(device)
                w = w.to(device)
                
                idx = [bn for bn in range(x.size(0)) if cost < random.random()]
                
                x = x.clone()
                if not self.teacher_force and (i > 0): # Never force
                    x[:, :self.outsize] = next_x.detach() # Always use predicted answer
                elif len(idx) and (i > 0): # Conditional force
                    x[idx, :self.outsize] = next_x[idx].detach() # Substitute in some predicted answers 
                else: # Always force
                    pass 
                                  
                next_x = self.model(x)
                y_true.append(y)
                y_pred.append(next_x)
                weights.append(w)
                
            loss = self.criterion.train(y_true, y_pred, weights)  
            epoch_losses["loss"].append(loss.item())
            
            # backprop after experiment
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            # update tqdm
            to_print = "Epoch {} training loss: {:.3f}".format(
                epoch, np.mean(epoch_losses["loss"])
            )
            batch_group_generator.set_description(to_print)
            batch_group_generator.update()
            
            if batch_idx > 0 and (batch_idx % steps) == 0:
                break
        #self.model = model_clone
        return np.mean(epoch_losses["loss"])
            
    def test(self, epoch):

        self.model.eval()
        batches_per_epoch = int(np.ceil(self.valid_gen.__len__() / self.batch_size))

        with torch.no_grad():

            batch_group_generator = tqdm(
                enumerate(range(len(self.valid_gen.experiment_subset))),
                total=len(self.valid_gen.experiment_subset), 
                leave=True
            )
            
            epoch_losses = {"loss": []}
            for batch_idx, exp in batch_group_generator:
                y_true, y_pred, weights = [], [], []
                for i, (x,y,w) in enumerate(self.valid_gen.__getitem__(exp)):
                    x = x.view(1, x.size(0)).to(device)
                    y = y.view(1, y.size(0)).to(device)
                    if i > 0:
                        x[:, :self.outsize] = next_x
                    next_x = self.model(x)
                    y_true.append(y)
                    y_pred.append(next_x)
                    
                loss = self.criterion.test(y_true, y_pred)   
                epoch_losses["loss"].append(loss.item())
                
                # update tqdm
                to_print = "Epoch {} validation loss: {:.3f}".format(
                    epoch, np.mean(epoch_losses["loss"])
                )
                batch_group_generator.set_description(to_print)
                batch_group_generator.update()
            
        return np.mean(epoch_losses["loss"]) 
    
    
    def train(self,
              scheduler,
              early_stopping,
              metrics_logger):
        
        logger.info(
            f"Training the model for up to {self.epochs} epochs starting at epoch {self.start_epoch}"
        )
        
        flag = isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau)
        
        for epoch in range(self.start_epoch, self.epochs):
            train_loss = self.train_one_epoch(epoch)
            test_loss = self.test(epoch)

            scheduler.step(test_loss if flag else epoch)
            early_stopping(epoch, test_loss, self.model, self.optimizer)

            # Write results to the callback logger 
            result = {
                "epoch": epoch,
                "train_loss": train_loss,
                "valid_loss": test_loss,
                "lr": early_stopping.print_learning_rate(self.optimizer),
                "teacher_forcing_score": self.tf_annealer(epoch) if self.teacher_force else 1.0
            }
            metrics_logger.update(result)

            if early_stopping.early_stop:
                logger.info("Early stopping")
                break

    def tf_annealer(self, epoch):
        return 1.0 * self.gamma ** epoch # 1/(1 + self.decay * epoch) 

In [21]:
trainer = BaseTrainer(
    model, 
    optimizer,
    train_data_set, 
    valid_data_set, 
    None, 
    None,
    device = device,
    **config["trainer"]
)

In [22]:
# Initialize LR annealing scheduler 
if "ReduceLROnPlateau" in config["callbacks"]:
    schedule_config = config["callbacks"]["ReduceLROnPlateau"]
    scheduler = ReduceLROnPlateau(trainer.optimizer, **schedule_config)
    #logging.info(
    #    f"Loaded ReduceLROnPlateau learning rate annealer with patience {schedule_config['patience']}"
    #)
elif "ExponentialLR" in config["callbacks"]:
    schedule_config = config["callbacks"]["ExponentialLR"]
    scheduler = ExponentialLR(trainer.optimizer, **schedule_config)
    #logging.info(
    #    f"Loaded ExponentialLR learning rate annealer with reduce factor {schedule_config['gamma']}"
    #)

# Early stopping
checkpoint_config = config["callbacks"]["EarlyStopping"]
early_stopping = EarlyStopping(**checkpoint_config)

# Write metrics to csv each epoch
metrics_logger = MetricsLogger(**config["callbacks"]["MetricsLogger"])

In [23]:
#torch.autograd.set_detect_anomaly(False)

In [24]:
trainer.train(scheduler, early_stopping, metrics_logger)

Epoch 24 validation loss: 0.291: 100%|██████████| 201/201 [01:03<00:00,  3.17it/s]


In [ ]:
f = pd.read_csv("results/9_29/exp1/training_log.csv")

In [ ]:
plt.semilogy(f["epoch"], f["train_loss"])
plt.semilogy(f["epoch"], f["valid_loss"])

plt.xlabel("Epoch", fontsize = 12)
plt.ylabel("Loss", fontsize = 12)

plt.legend(["train", "valid"])
plt.tight_layout()

### Predict with the best model

In [ ]:
model = DenseNet(**config["model"])

In [ ]:
model_dict = torch.load("results/9_29/row1/best.pt", 
                        map_location=lambda storage, loc: storage)
model.load_state_dict(model_dict["model_state_dict"])

In [ ]:
model.eval()

In [ ]:
# if is_cuda:
#     model = model.to(device)
device = "cpu"
model = model.to(device)

In [ ]:
loss, y_pred, y_true = [], [], []

for (x,y,w) in tqdm(valid_data_set.__getitem__(0)):
    x = x.view(1, x.size(0)).to(device)
    y = y.view(1, y.size(0)).to(device)
    w = w.to(device)
    
    next_x = model(x)
    y_pred.append(next_x)
    y_true.append(y)    

In [ ]:
loss, y_pred, y_true = [], [], []

for k, (x,y,w) in tqdm(enumerate(valid_data_set.__getitem__(0))):
    x = x.view(1, x.size(0)).to(device)
    y = y.view(1, y.size(0)).to(device)
    w = w.to(device)
    
    X = x
    #if k == 0:
    #    X = x
    
    next_x = model(X)
    #loss.append(RMSELoss()(next_x, y[:,t]))
    
    y_pred.append(next_x)
    y_true.append(y) 
    
    #X[:, :model.output_size] = next_x
    
# #loss = torch.mean(torch.stack(loss))
# preds = torch.stack(preds, axis = -1).permute(0, 2, 1)

In [ ]:
def quantities(Y):
    a, b, c = [], [], []
    for y in Y:
        y = y[0, :]
        prec = y[0]
        gas = y[1:15].sum(-1)/14
        aero = y[15:].sum(-1)/14
        a.append(prec.item())
        b.append(gas.item())
        c.append(aero.item())
    return a, b, c

In [ ]:
prec_pred, gas_pred, aero_pred = quantities(y_pred)
prec_true, gas_true, aero_true = quantities(y_true)

In [ ]:
plt.plot(range(len(prec_pred)), prec_pred)
plt.plot(range(len(prec_pred)), prec_true)

plt.legend(["Predicted", "True"])

plt.ylabel("Precursor/Gas/Aerosol")
plt.xlabel("Time steps")

plt.ylim([0.0, 1.01])

In [ ]:
plt.plot(range(len(prec_pred)), gas_pred)
plt.plot(range(len(prec_pred)), gas_true)

plt.legend(["Predicted", "True"])

plt.ylabel("Precursor/Gas/Aerosol")
plt.xlabel("Time steps")

plt.ylim([0.0, 1.01])

In [ ]:
plt.plot(range(len(prec_pred)), aero_pred)
plt.plot(range(len(prec_pred)), aero_true)

plt.legend(["Predicted", "True"])

plt.ylabel("Precursor/Gas/Aerosol")
plt.xlabel("Time steps")

plt.ylim([0.0, 1.01])